### Import function

In [15]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import json
from dsmlibrary.datanode import DataNode

### Upload web ที่ทำการ Scrapy และทำการ Scrapy ข้อมูล

In [16]:
data = requests.get('https://www.kasikornresearch.com/th/analysis/Pages/index.aspx?s=สมุนไพร')
data

<Response [200]>

In [17]:
s = BeautifulSoup(data.content, 'html.parser')

In [18]:
publish_date = []
retrieve_date = []
title = []
content = []
source_url = []

In [19]:
url_pre = 'https://www.kasikornresearch.com/'

for news in s.find_all('div', {'class': 'meta'}):
    link = url_pre + news.find('a')['href']
    source_url.append(link)
    
    topic = news.find('a').text
    title.append(topic)
    
    pub = news.find('p').text
    publish_date.append(pub)
    
    retrieve = str(datetime.date.today())
    retrieve_date.append(retrieve)

# print(publish_datetime)
# print(title) 
# print(source_url)

In [20]:
for c in source_url:
    so = requests.get(c)
    no = BeautifulSoup(so.content, 'html.parser')
    
    for n in no.find_all('div', {'class', 'art-content entrycontent'}):
        a = n.text
        a = a.strip()
        a = a.replace('\u200b', '')
        a = a.replace('\xa0', '')
        a = a.replace('\r', '')
        a = a.replace('\n', '')
        a = a.replace('Scan QR Codeหมายเหตุรายงานวิจัยฉบับนี้จัดทำเพื่อเผยแพร่ทั่วไป โดยจัดทำขึ้นจากแหล่งข้อมูลต่างๆ ที่น่าเชื่อถือ แต่บริษัทฯ มิอาจรับรองความถูกต้อง ความน่าเชื่อถือ หรือความสมบูรณ์เพื่อใช้ในทางการค้าหรือประโยชน์อื่นใด บริษัทฯ อาจมีการเปลี่ยนแปลงข้อมูลได้ตลอดโดยไม่ต้องแจ้งให้ทราบล่วงหน้า ทั้งนี้ผู้ใช้ข้อมูลต้องใช้ความระมัดระวังในการใช้ข้อมูลต่างๆ ด้วยวิจารณญาณตนเองและรับผิดชอบในความเสี่ยงเองทั้งสิ้น บริษัทฯ จะไม่รับผิดต่อผู้ใช้หรือบุคคลใดในความเสียหายใดจากการใช้ข้อมูลดังกล่าว ข้อมูลในรายงานฉบับนี้จึงไม่ถือว่าเป็นการให้ความเห็น หรือคำแนะนำในการตัดสินใจทางธุรกิจ แต่อย่างใดทั้งสิ้น', '')
        content.append(a)

# print(content)

In [21]:
df = pd.DataFrame([title, publish_date, content, source_url, retrieve_date]).transpose()
df.columns = ['title', 'publish_date', 'content', 'source_url', 'retrieve_date']
df

,title,publish_date,content,source_url,retrieve_date
0,ฟ้าทะลายโจร ยกระดับสมุนไพรไทย สู่อุตสาหกรรมยา,2 มีนาคม 2564,Caption Video,https://www.kasikornresearch.com//th/analysis/...,2022-11-19
1,สมุนไพรไทย โอกาสธุรกิจที่ต้องคว้าไว้,25 กุมภาพันธ์ 2564,Caption Video,https://www.kasikornresearch.com//th/analysis/...,2022-11-19
2,สมุนไพรไทย โอกาสธุรกิจ...ที่ต้องคว้าไว้,25 พฤศจิกายน 2563,จากกระแสนิยมดูแลรักษาสุขภาพโดยวิถีธรรมชาติแ...,https://www.kasikornresearch.com//th/analysis/...,2022-11-19
3,ฟ้าทะลายโจร ยกระดับการผลิตสมุนไพรไทย สู่อุตสาห...,20 เมษายน 2563,ฟ้าทะลายโจร ถูกพูดถึงมากขึ้นในช่วงของการระบาดข...,https://www.kasikornresearch.com//th/analysis/...,2022-11-19
4,ถึงเวลาโกยเงิน SME รุกตลาดสมุนไพร,18 มีนาคม 2562,สมุนไพรถือเป็นดาวเด่นในแวดวงธุรกิจเพื่อสุขภาพแ...,https://www.kasikornresearch.com//th/analysis/...,2022-11-19


In [22]:
df_json = df.to_dict('records')

In [23]:
print(f'Total news : {len(df_json)}')

Total news : 5


### เรียกใช้ datanode และUpload ข้อมูลลง DataPlatform

In [24]:
with open('HERB_KASIKORNRESEARCH.json', 'w', encoding='utf8') as f:
    json.dump(df_json, f)

In [25]:
dir_raw_id = 71

In [26]:
datanode = DataNode()

Init DataNode sucessful!


In [27]:
datanode.upload_file(directory_id=dir_raw_id, file_path='HERB_KASIKORNRESEARCH.json')

  0%|          | 0.00/52.0k [00:00<?, ?B/s]

{'id': 886,
 'msg': 'file HERB_KASIKORNRESEARCH.json create sucessful',
 'path': 'kanlaya/DSI310-17/rawData/HERB_KASIKORNRESEARCH.json',
 'uploaded': True}